# 🧩 7º Fórum da Guilda de Engenharia de Software
## Entenda Clean Architecture De Vez

### 🎯 Objetivo
Apresentar os fundamentos da Clean Architecture, seus principais componentes e promover uma conversa aberta sobre dificuldades, interpretações equivocadas e adaptações pragmáticas.

### 📚 Estrutura
- Introdução expositiva com exemplos visuais e trechos de código (~20 min)
- Discussão guiada com perguntas (~30–40 min)

### ❓ Por que Clean Architecture?
- Separar regras de negócio de detalhes técnicos.
- Ajudar a evoluir sistemas de forma sustentável.
- Tornar mudanças menos custosas (ex: mudar banco, UI ou framework).

### 🧭 Regra de Ouro: Dependências apontam para o domínio
```
[ Frameworks & Drivers ] -> [ Interface Adapters ] -> [ Application Business Rules ] -> [ Enterprise Business Rules ]
```
- As camadas mais externas conhecem as internas, mas não o contrário.

### 🧅 Camadas

![Clean Architecture](imagens/CleanArchitecture.jpg)

![The Clean Architecture Cone](imagens/TheCleanArchitectureCone.jpeg)

### 🧱 Componentes Principais


#### 🧩 Entities
Lógica de negócio pura, sem dependências externas.
```python
class Pedido:
    def __init__(self):
        self.itens = []

    def adicionar_item(self, item):
        self.itens.append(item)

    def calcular_total(self):
        return sum(item.preco for item in self.itens)
```

#### 🧠 UseCases
Orquestra ações e regras de negócio.
```python
class CriarPedido:
    def __init__(self, repositorio):
        self.repositorio = repositorio

    def executar(self, dados):
        pedido = Pedido()
        for item in dados['itens']:
            pedido.adicionar_item(item)
        self.repositorio.salvar(pedido)
        return pedido
```

#### 🖼️ Presenter
Formata a saída para o consumidor da aplicação (ex: JSON).

🎯 O que o Presenter faz numa API?

Exemplos de responsabilidades do Presenter:

* Traduzir entidades e objetos de domínio em DTOs ou dicts JSON-compatíveis
* Adaptar formatos (ex: Decimal → str, datetime → ISO string)
* Filtrar ou reorganizar dados para a resposta
* Traduzir enums, erros ou códigos internos para nomes amigáveis
* Preparar resposta paginada, envelopes (data, meta, links, etc)

#### 🧱 Exemplo prático

UseCase (devolve dados estruturados internos)

```python
class ConsultarSaldoUseCase:
    def executar(self, conta_id):
        conta = self.conta_repo.buscar(conta_id)
        return {"saldo": conta.saldo, "moeda": conta.moeda}
```

Presenter (transforma em algo que a API REST entende)

```python
class SaldoPresenter:
    def apresentar(self, dados):
        return {
            "data": {
                "saldo": f"{dados['saldo']:.2f}",
                "moeda": dados["moeda"]
            }
        }
```

#### 🎨 Exemplo de Presenter com enriquecimento


Presenter que formata os dados para a interface externa com informações adicionais:
- Nome do cliente (via repositório)
- Link de rastreamento (via gateway)

```python
class PedidoPresenter:
    def __init__(self, cliente_repo, entrega_gateway):
        self.cliente_repo = cliente_repo
        self.entrega_gateway = entrega_gateway

    def apresentar(self, pedido):
        cliente = self.cliente_repo.buscar_por_id(pedido.cliente_id)
        rastreamento = self.entrega_gateway.link_rastreamento(pedido.id)

        return {
            "id": pedido.id,
            "total": pedido.valor_total(),
            "status_pagamento": pedido.status_pagamento,
            "cliente": cliente.nome,
            "rastreamento": rastreamento
        }
```

#### 🛠️ Service
Executar uma ação ou efeito colateral controlado

```python
class EmailService:
    def __init__(self, client, presenter):
        self.client = client
        self.presenter = presenter

    def enviar_confirmacao(self, pedido):
        corpo = self.presenter.formatar_mensagem(pedido)
        self.client.send(to=pedido.cliente.email, subject="Pedido confirmado", body=corpo)
```

#### 🗃️ Repository
Acesso a dados persistidos (banco, arquivos, etc.)
```python
class PedidoRepository:
    def buscar_por_id(self, pedido_id):
        # Consulta no banco e retorna um objeto de domínio
        ...

    def salvar(self, pedido):
        # Persiste o objeto no banco
        ...
```

#### 🧩 Adapter

Conecta o mundo externo ao sistema, traduzindo a entrada (ex: HTTP).

```python
# DTO de entrada
class ItemDTO(BaseModel):
    produto_id: str
    quantidade: int

class CriarPedidoDTO(BaseModel):
    cliente_id: str
    itens: List[ItemDTO]

# Controller (Adapter)
@app.post("/pedidos")
def criar_pedido(dto: CriarPedidoDTO):
    usecase = CriarPedidoUseCase(pedido_repo)
    resultado = usecase.executar(dto.dict())  # ou transformar em Entity
    return PedidoPresenter().apresentar(resultado)
```

#### 🌐 Resource / Controller
Ponto de entrada da aplicação.
```python
@app.route('/pedido', methods=['POST'])
def criar_pedido():
    dados = request.json
    usecase = CriarPedido(repo)
    pedido = usecase.executar(dados)
    return jsonify(apresentar_pedido(pedido))
```

### 💬 Perguntas para Discussão
1. Em que ponto vocês mais se confundem ao tentar aplicar Clean Architecture?
2. Em que situações parece “overengineering”? Já passou por isso?
3. Até onde você acha que o UseCase deve ir? E o Presenter?
4. Você já precisou trocar algo grande (como UI, banco ou API externa)? Teria sido mais fácil com essa arquitetura?
5. Como aplicar Clean Architecture num projeto pequeno ou legado?

### Referências

1. **Martin, R.C.**  
   [*The Clean Architecture*](https://blog.cleancoder.com/uncle-bob/2012/08/13/the-clean-architecture.html).  
   *8th Light Blog* (2012).  
   – Conceitos originais da Clean Architecture e relação com microsserviços.

2. **Microsoft Docs**  
   [*Common web application architectures*](https://learn.microsoft.com/en-us/dotnet/architecture/modern-web-apps-azure/common-web-application-architectures).  
   *Ebook: Architect Modern Web Apps with ASP.NET Core*.  
   – Explicação de Clean Architecture, camadas em projetos .NET, vantagens em testes e DI.

3. **Merwan Chinta**  
   [*Code in Clean vs Traditional Layered Architecture (.NET)*](https://medium.com/@merwanchinta/code-in-clean-vs-traditional-layered-architecture-net-dfcfd1d18934).  
   *Medium.com* (2024).  
   – Comparação entre arquitetura em camadas tradicional e Clean Architecture em .NET.

4. **ISE / Microsoft DevBlog**  
   [*Next-Level Boilerplate: .NET Clean Architecture*](https://devblogs.microsoft.com/ise/next-level-boilerplate-dotnet-clean-architecture/).  
   *Microsoft Dev Blog* (2023).  
   – Exemplo real usando Clean Arch em .NET, vantagens observadas e contexto de uso ideal.

5. **Reddit /r/dotnet**  
   [*“A Arquitetura Limpa realmente é necessária?”*](https://www.reddit.com/r/dotnet/comments/14qbgla/clean_architecture_criticisms_and_when_not_to_use/).  
   *Reddit thread* (2023).  
   – Debate com prós e contras, contexto de startups vs enterprise, opiniões de devs experientes sobre quando usar.

6. **Sebastian Buczyński**  
   [*Python & the Clean Architecture in 2021*](https://breadcrumbscollector.tech/python-clean-architecture-in-2021/).  
   *BreadcrumbsCollector.tech* (2021).  
   – Lições aprendidas ao implementar Clean Arch em Python, evitando overengineering e usando frameworks modernos.

7. **MSC – Dev.to**  
   [*Practical Clean Architecture in TypeScript, Rust & Python*](https://dev.to/msc/clean-architecture-in-typescript-rust-and-python-5c1a).  
   *Dev.to* (2022).  
   – Projeto demonstrando Clean Arch multilinguagem, isolando lógica de negócio de infraestrutura.

8. **Chris (Dev.to)**  
   [*Stop Overengineering in the Name of Clean Architecture*](https://dev.to/chris/stop-overengineering-in-the-name-of-clean-architecture-2cfa).  
   *Dev.to* (2023).  
   – Alerta sobre abstrações desnecessárias e exemplos de código exageradamente “limpo”.

9. **Steve Smith (Ardalis)**  
   [*Clean Architecture Sucks*](https://ardalis.com/clean-architecture-sucks-sometimes/).  
   *Ardalis.com* (2024).  
   – Estudo de caso de falha por má implementação e conselho sobre adequação da arquitetura ao time.

10. **Exemplo GitHub**  
   [*DDD + Clean Architecture PetClinic*](https://github.com/ardalis/ddd-clean-architecture/tree/main/src/PetClinic).  
   *GitHub.com* (2020).  
   – Demonstração de módulos de domínio em aplicação de clínica veterinária usando Clean Architecture.
